In [3]:
!git clone https://github.com/BNLNLP/PPI-Relation-Extraction.git

Cloning into 'PPI-Relation-Extraction'...
remote: Enumerating objects: 899, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 899 (delta 7), reused 14 (delta 6), pack-reused 884
Receiving objects: 100% (899/899), 20.93 MiB | 14.43 MiB/s, done.
Resolving deltas: 100% (499/499), done.


In [4]:
!pip install -r /kaggle/working/PPI-Relation-Extraction/requirements.txt

ERROR: Could not find a version that satisfies the requirement torch==1.10.2 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1, 2.1.2)
ERROR: No matching distribution found for torch==1.10.2


## Weights and Biases (wandb) installation and login

In [5]:
import wandb
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

# Running inference

In [11]:
%cd "/kaggle/working/"

/kaggle/working


In [19]:
!python run_re.py \
  --model_list dmis-lab/biobert-base-cased-v1.1 \
  --task_name "re" \
  --dataset_dir "/kaggle/working/PPI-Relation-Extraction/datasets" \
  --dataset_name "/kaggle/working/PPI-Relation-Extraction/datasets/PPI/original/HPRD50" \
  --output_dir "/kaggle/working/PPI-Relation-Extraction/Outputs" \
  --do_train \
  --do_predict \
  --seed 1 \
  --remove_unused_columns False \
  --save_steps 100000 \
  --per_device_train_batch_size 16 \
  --per_device_eval_batch_size 32 \
  --num_train_epochs 10 \
  --optim "adamw_torch" \
  --learning_rate 5e-05 \
  --warmup_ratio 0.0 \
  --weight_decay 0.0 \
  --relation_representation "EM_entity_start" \
  --use_context "attn_based" \
  --do_cross_validation True \
  --overwrite_cache \
  --overwrite_output_dir

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"



 Initial Output Dir at first instance:/kaggle/working/PPI-Relation-Extraction/Outputs






 Initial Output Dir in for loop:/kaggle/working/PPI-Relation-Extraction/Outputs



*** No. of datasets: 1 ***
*** No. of datasets: 2 ***
*** No. of datasets: 3 ***
*** No. of datasets: 4 ***
*** No. of datasets: 5 ***
*** No. of datasets: 6 ***
*** No. of datasets: 7 ***
*** No. of datasets: 8 ***
*** No. of datasets: 9 ***
*** No. of datasets: 10 ***
[INFO|configuration_utils.py:715] 2023-12-27 18:34:33,423 >> loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--dmis-lab--biobert-base-cased-v1.1/snapshots/924f12e0c3db7f156a765ad53fb6b11e7afedbc8/config.json
[INFO|configuration_utils.py:775] 2023-12-27 18:34:33

In [11]:
!wget /kaggle/working/PPI-Relation-Extraction/datasets/PPI/original/HPRD50/dmis-lab/biobert-base-cased-v1.1/EM_entity_start_ac/pytorch_model.bin

/kaggle/working/PPI-Relation-Extraction/datasets/PPI/original/HPRD50/dmis-lab/biobert-base-cased-v1.1/EM_entity_start_ac/pytorch_model.bin: Scheme missing.


### **--All Hyperparameters--**

In [ ]:
!mv "/kaggle/working/PPI-Relation-Extraction/datasets/EU-ADR_BioBERT (train & validation)" "/kaggle/working/PPI-Relation-Extraction/datasets/EU-ADR_BioBERT"

In [6]:
%cd /kaggle/working/PPI-Relation-Extraction/src/relation_extraction

/kaggle/working/PPI-Relation-Extraction/src/relation_extraction


In [18]:
%%writefile dataset_utils.py
import os
import sys
import re
import pickle
import json
import pandas as pd
import numpy as np
from datasets import ClassLabel, load_dataset, load_metric, Dataset, DatasetDict, concatenate_datasets
from transformers import BertTokenizerFast, RobertaTokenizerFast

import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

import random

def extract_json_objects(file_path):
    json_objects = []
    with open(file_path,"r") as file:
        for line in file:
            try:
                json_object = json.loads(line)
                json_objects.append(json_object)
            except Exception as e:
                print(f"Error parsing line {file.tell() - len(line)}: {e}")
    return json_objects

def read_dataset(dataset_num=0, task_name=None, data_args=None):
    data_files = {}
    if data_args.train_file is not None and data_args.test_file is not None:
        data_files["train"] = data_args.train_file
        data_files["test"] = data_args.test_file
        print(f"\nTrain File: {data_args.train_file}. Test File: {data_args.test_file}\n")
    else:
        data_dir = os.path.join(data_args.dataset_dir, data_args.dataset_name)
        # data_files["train"] = os.path.join(data_dir, 'train_' + str(dataset_num) + '.json')
        training_data = os.path.join(data_dir, 'train_' + str(dataset_num) + '.json')
        data_files["test"] = os.path.join(data_dir, 'test_' + str(dataset_num) + '.json')
        print(f"\nTrain File: {dataset_num}. Test File: {dataset_num}\n")
        if os.path.isfile(os.path.join(data_dir, 'dev_' + str(dataset_num) + '.json')):
            data_files["validation"] = os.path.join(data_dir, 'dev_' + str(dataset_num) + '.json')
        else:
            
            
            # Open a json file
            json_objects = extract_json_objects(training_data)
            data = [item for sublist in json_objects for item in sublist.items()]
            # Load all the contents to a dictionary
            sorted_data = sorted(data)
            
            # Sort and shuffle the data
            random.shuffle(sorted_data)
            
            # Extract the remaining data into a dictionary, data_files under the key "validation"
            training_data_cutoff = int(0.8 * len(sorted_data))
            training_data = dict(sorted_data[:training_data_cutoff])
            validation_data = dict(sorted_data[training_data_cutoff:])
            
            data_files["train"] = training_data
            data_files["validation"] = validation_data
            
            print(f"\nData Files being sent in for eval set:{data_files['train']}.\n")
            
    extension = data_files["train"].split(".")[-1]
    print(f"\nExtension: {extension}\n")
    print(f"\nData Files: {data_files}\n")
    
    return load_dataset(extension, data_files=data_files)
    
    
def tokenize_and_set_relation_labels(examples, tokenizer, padding, max_seq_length, relation_representation, use_context):

    if 'text' in examples:
        if relation_representation.startswith('EM'):
            token_key = 'text_with_entity_marker'
        else:
            token_key = 'text'
        
        tokenized_inputs = tokenizer(
            examples[token_key],
            padding=padding,
            truncation=True,
            max_length=max_seq_length,
        )
    elif 'tokens' in examples:
        if relation_representation.startswith('EM'):
            token_key = 'tokens_with_marker'
        else:
            token_key = 'tokens'
        
        tokenized_inputs = tokenizer(
            examples[token_key],
            padding=padding,
            truncation=True,
            max_length=max_seq_length,
            # We use this argument because the texts in our dataset are lists of words.
            is_split_into_words=True,	
        )
    else:
        raise Exception("There is no tokens element in the data!!")

    labels = []
    relations = []
    
    tokens_seq = []
    tokens_to_ignore = []

    # Most data has a single relation per example, but some data such as SciERC has multiple relations in a sentence.
    for i, rel_list in enumerate(examples['relation']):
        label_ids = []
        relation_spans = []
        predicate_spans = []
        ent_types = []
        
        for rel in rel_list:
            if 'text' in examples:
                
                # ref: https://www.lighttag.io/blog/sequence-labeling-with-transformers/example
                # ref: https://github.com/huggingface/transformers/issues/9326
                def get_token_idx(char_idx):
                    while True:
                        # if it's the last index, return the last token.
                        if char_idx == len(examples[token_key][i]):
                            return len(tokenized_inputs[i]) - 1
                        
                        token_idx = tokenized_inputs.char_to_token(batch_or_char_index=i, char_index=char_idx)
                        # Whitespaces have no token and will return None.
                        if token_idx is not None:
                            return token_idx
                        
                        char_idx += 1
                        # debug
                        #if char_idx == len(examples[token_key][i]):
                        #	raise Exception("End token not found: " f"{tokenizer.convert_ids_to_tokens(tokenized_inputs['input_ids'][i])}")

                e1_span_idx_list, e2_span_idx_list = [], []
                
                if relation_representation.startswith('EM'):
                    e1_idx = rel['entity_1_idx_in_text_with_entity_marker']
                    e2_idx = rel['entity_2_idx_in_text_with_entity_marker']
                    
                    ## TODO: remove this!! the first token is used for separate tokens.
                    if np.asarray(e1_idx).ndim > 1 or np.asarray(e2_idx).ndim > 1:
                        raise Exception("For now, entity marker representations do not support separate entities.")
                else:
                    e1_idx = rel['entity_1_idx']
                    e2_idx = rel['entity_2_idx']
                
                # Some dataset (e.g., BioInfer) has entities consisting of separate tokens.
                # To match the dimension to separate entities, add a dimension for single entities. 
                e1_idx = [e1_idx] if np.asarray(e1_idx).ndim == 1 else e1_idx
                e2_idx = [e2_idx] if np.asarray(e2_idx).ndim == 1 else e2_idx
                
                for e1_s, e1_e in e1_idx:
                    e1_span_s = get_token_idx(e1_s)
                    e1_span_e = get_token_idx(e1_e)
                    e1_span_idx_list.append((e1_span_s, e1_span_e))
                
                for e2_s, e2_e in e2_idx:
                    e2_span_s = get_token_idx(e2_s)
                    e2_span_e = get_token_idx(e2_e)
                    e2_span_idx_list.append((e2_span_s, e2_span_e))
            
            entity_1_type_id = rel['entity_1_type_id']
            entity_2_type_id = rel['entity_2_type_id']

            label_ids.append(rel['relation_id'])
            relation_spans.extend([e1_span_idx_list, e2_span_idx_list])
            ent_types.extend([entity_1_type_id, entity_2_type_id])
            
        labels.append(label_ids)
        relations.append(relation_spans)

        if use_context == "attn_based":
            input_tokens = tokenized_inputs.tokens(batch_index=i)
            
            entity_indice = []
            for r_s in relation_spans:
                for span_s, span_e in r_s:
                    entity_indice.extend(list(range(span_s, span_e)))
            
            entity_indice = list(set(entity_indice))

            tokens_seq.append([1 if tok.startswith('##') else 0 for tok in input_tokens])
            tokens_to_ignore.append([-100 if re.search('[a-zA-Z0-9]', tok) == None or \
                                             #tok in tokenizer.all_special_tokens or \
                                             tok in list(set(tokenizer.all_special_tokens) - set(tokenizer.additional_special_tokens)) or \
                                             idx in entity_indice \
                                          else 0 for idx, tok in enumerate(input_tokens)])
            
            tokenized_inputs['tokens_seq'] = tokens_seq
            tokenized_inputs['tokens_to_ignore'] = tokens_to_ignore
    
    tokenized_inputs['labels'] = labels
    tokenized_inputs['relations'] = relations
    
    return tokenized_inputs


def featurize_data(dataset, tokenizer, padding, max_seq_length, relation_representation, use_context):
    convert_func_dict = tokenize_and_set_relation_labels
    
    if use_context == "attn_based":
        columns = ['input_ids', 'attention_mask', 'labels', 'token_type_ids', 'relations', 'tokens_seq', 'tokens_to_ignore']
    else:
        columns = ['input_ids', 'attention_mask', 'labels', 'token_type_ids', 'relations']

    features = {}
    for phase, phase_dataset in dataset.items():
        features[phase] = phase_dataset.map(
            convert_func_dict,
            fn_kwargs={'tokenizer': tokenizer,
                       'padding': padding,
                       'max_seq_length': max_seq_length,
                       'relation_representation': relation_representation,
                       'use_context': use_context},
            batched=True,
            load_from_cache_file=False,
        )
        
        features[phase].set_format(
            #type="torch",
            type=None,
            columns=columns,
        )
    
    return features


Overwriting dataset_utils.py


In [8]:
%%writefile run_re.py
import os
import sys
import logging
import shutil
import json
import numpy as np
from dataclasses import dataclass, field
from typing import List, Union, Dict, Optional, Tuple

import torch
import torch.nn as nn
from torch.utils.data.dataloader import DataLoader
#from torch.utils.data.distributed import DistributedSampler
from torch.utils.data.sampler import RandomSampler

import datasets
from datasets import load_dataset, load_metric

import transformers
from transformers import (
    AutoConfig,
    AutoModelForTokenClassification,
    AutoTokenizer,
    BertTokenizerFast,
    RobertaTokenizerFast,
    DebertaV2Tokenizer,
    DebertaTokenizerFast,
    DataCollatorForTokenClassification,
    HfArgumentParser,
    PreTrainedModel,
    PreTrainedTokenizerFast,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    EvalPrediction,
    set_seed,
)
from transformers.data.data_collator import DataCollator, InputDataClass
from transformers.trainer_utils import get_last_checkpoint, is_main_process
from transformers.utils import check_min_version

from dataset_utils import *
from model import BertForRelationClassification
from data_collator import DataCollatorForRelationClassification


dataset_list = ["ChemProt_BLURB", "DDI_BLURB", "GAD_BLURB", "EU-ADR_BioBERT",
                "AImed", "BioInfer", "HPRD50", "IEPA", "LLL",
                "Typed_PPI",
                "P-putida", "P-species"]

dataset_max_seq_length = {
    #"ChemProt_BLURB": 256, # some samples (count: 11) are longer than 256 tokens.
    #"DDI_BLURB": 256, # many samples are longer than 256 tokens. 
    #"GAD_BLURB": 128, # some samples (count: 3) are longer than 128 tokens when BioBERT is used.
    #"EU-ADR_BioBERT": 128, # some samples (count: 14) are longer than 128 tokens.
}

dataset_special_tokens = {
    "ChemProt_BLURB": ["@GENE$", "@CHEMICAL$", "@CHEM-GENE$"],
    "DDI_BLURB": ["@DRUG$", "@DRUG-DRUG$"],
    "GAD_BLURB": ["@GENE$", "@DISEASE$"],
    "EU-ADR_BioBERT": ["@GENE$", "@DISEASE$"],
}

entity_marker_special_tokens = {
    "EM": ["[E1]", "[/E1]", "[E2]", "[/E2]", "[E1-E2]", "[/E1-E2]"],
    "ChemProt_BLURB": ["[GENE]", "[/GENE]", "[CHEM]", "[/CHEM]", "[CHEM-GENE]", "[/CHEM-GENE]"],
    "DDI_BLURB": ["[DRUG]", "[DRUG-DRUG]"],
    "GAD_BLURB": ["[GENE]", "[/GENE]", "[DISEASE]", "[/DISEASE]"],
}

logger = logging.getLogger(__name__)


@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    # [START][GP] - input parameter for a list of models. 04-11-2021
    model_name_or_path: str = field(
        default='bert-base-cased',
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    model_list: List[str] = field(
        default_factory=lambda: ['bert-base-cased', 
                                 'bert-large-cased', 
                                 'dmis-lab/biobert-base-cased-v1.1', 
                                 'dmis-lab/biobert-large-cased-v1.1'],
        metadata={"help": "a list of models."},
    )
    # [END][GP] - input parameter for a list of models. 04-11-2021

    config_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained config name or path if not the same as model_name"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None,
        metadata={"help": "Where do you want to store the pretrained models downloaded from huggingface.co"},
    )
    model_revision: str = field(
        default="main",
        metadata={"help": "The specific model version to use (can be a branch name, tag name or commit id)."},
    )
    use_auth_token: bool = field(
        default=False,
        metadata={
            "help": "Will use the token generated when running `transformers-cli login` (necessary to use this script "
            "with private models)."
        },
    )
    # [START][GP] - added do_lower_case parameter for tokenizer. 04-07-2021
    do_lower_case: bool = field(
        default=False,
        metadata={
            "help": "Whether to lowercase words or not. Basically, this option follows model's config, "
            "but, some models (e.g., BioBERT cased) needs to be explicitly set."
        },
    )
    # [END][GP] - added do_lower_case parameter for tokenizer. 04-07-2021


@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """
    
    task_name: Optional[str] = field(default="re", metadata={"help": "The name of the task."})

    dataset_name: Optional[str] = field(
        default=None, metadata={"help": "The name of the dataset to use (via the datasets library)."}
    )
    dataset_config_name: Optional[str] = field(
        default=None, metadata={"help": "The configuration name of the dataset to use (via the datasets library)."}
    )
    max_seq_length: int = field(
        default=512,
        metadata={
            "help": "The maximum total input sequence length after tokenization. Sequences longer "
            "than this will be truncated, sequences shorter will be padded."
        },
    )
    train_file: Optional[str] = field(
        default=None, metadata={"help": "The input training data file (a JSON file)."}
    )
    validation_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input evaluation data file to evaluate on (a JSON file)."},
    )
    test_file: Optional[str] = field(
        default=None,
        metadata={"help": "An optional input test data file to predict on (a JSON file)."},
    )
    overwrite_cache: bool = field(
        default=False, metadata={"help": "Overwrite the cached training and evaluation sets"}
    )
    preprocessing_num_workers: Optional[int] = field(
        default=None,
        metadata={"help": "The number of processes to use for the preprocessing."},
    )
    pad_to_max_length: bool = field(
        default=False,
        metadata={
            "help": "Whether to pad all samples to model maximum sentence length. "
            "If False, will pad the samples dynamically when batching to the maximum length in the batch. More "
            "efficient on GPU but very bad for TPU."
        },
    )
    max_train_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of training examples to this "
            "value if set."
        },
    )
    max_eval_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of evaluation examples to this "
            "value if set."
        },
    )
    max_test_samples: Optional[int] = field(
        default=None,
        metadata={
            "help": "For debugging purposes or quicker training, truncate the number of test examples to this "
            "value if set."
        },
    )

    # [START][GP] - data parameters.
    dataset_dir: Optional[str] = field(
        default=None, metadata={"help": "The path to the parent directory of all datasets."},
    )
    relation_types: str = field(
        default=None, metadata={"help": "Relation type file (name and id)."},
    )
    relation_representation: str = field(
        default="STANDARD_mention_pooling",
        metadata={"help": "vairous relation representations from [2019] Matching the Blanks: Distributional Similarity for Relation Learning. "
                          "Largely, the representations are divided into standard and entity markers. "
                          "Options: "
                          "1) standard: STANDARD_cls_token, "
                          "				STANDARD_mention_pooling, "
                          "2) entity markers (EM): EM_cls_token, "
                          "						   EM_mention_pooling, "
                          "						   EM_entity_start, "
                          " * for poolings, max pooling is used. "},
    )
    use_context: str = field(
        default=None,
        metadata={"help": "Here, context indicates tokens related to entities' relational information. "
                          "The context is appended to relation representations.  "
                          "Options: "
                          "1) attn_based: context based on attention probability calculation, "
                          "2) local: local context (tokens between the two entities) "},
    )
    # [END][GP] - data parameters.

    # [START][GP] - cross-validation parameters.
    do_cross_validation: Optional[bool] = field(
        default=False, 
        metadata={"help": "Whether to use cross-validation for evaluation."}
    )
    num_of_folds: Optional[int] = field(
        default=10, 
        metadata={"help": "The number of folds for the cross-validation."}
    )
    ratio: Optional[str] = field(
        default='80-10-10', 
        metadata={"help": "train/dev/test ratio: 80-10-10, 70-15-15, 60-20-20"}
    )
    # [END][GP] - cross-validation parameters.
    
    save_predictions: Optional[bool] = field(
        default=False, 
        metadata={"help": "Whether to save predictions along with ground truth labels. It's usually for debugging purpose."}
    )
    
    def __post_init__(self):
        if self.dataset_name is None and self.train_file is None and self.validation_file is None:
            raise ValueError("Need either a dataset name or a training/validation file.")
        elif self.dataset_name is not None:
            d_name = self.dataset_name.rsplit('/', 1)[1] if len(self.dataset_name.rsplit('/', 1)) > 1 else self.dataset_name
            if d_name not in dataset_list:
                raise ValueError("Unknown dataset, you should pick one in " + ", ".join(dataset_list) + ". Or, add your dataset to the dataset list.")
        else:
            if self.train_file is not None:
                extension = self.train_file.split(".")[-1]
                assert extension == "json", "`train_file` should be a json file."
            if self.validation_file is not None:
                extension = self.validation_file.split(".")[-1]
                assert extension == "json", "`validation_file` should be a json file."
            if self.test_file is not None:
                extension = self.test_file.split(".")[-1]
                assert extension == "json", "`test_file` should be a json file."


def main():
    # See all possible arguments in src/transformers/training_args.py
    # or by passing the --help flag to this script.
    # We now keep distinct sets of args, for a cleaner separation of concerns.

    parser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
    if len(sys.argv) == 2 and sys.argv[1].endswith(".json"):
        # If we pass only one argument to the script and it's the path to a json file,
        # let's parse it to get our arguments.
        model_args, data_args, training_args = parser.parse_json_file(json_file=os.path.abspath(sys.argv[1]))
    else:
        model_args, data_args, training_args = parser.parse_args_into_dataclasses()

    # Setup logging
    # Yashfinul Haque -28/11/23
#     FORMAT = '%(asctime)s %(clientip)-15s %(user)-8s %(levelname)-8s %(message)s'

#     logging.basicConfig(
#         format=FORMAT,
#         datefmt="%m/%d/%Y %H:%M:%S",
#         level=logging.DEBUG,  # Set the logging level to DEBUG to capture all levels
#         handlers=[logging.StreamHandler(sys.stdout)],
#     )
    
#     log_level = training_args.get_process_log_level()
#     logger.setLevel(log_level)
#     datasets.utils.logging.set_verbosity(log_level)
#     transformers.utils.logging.set_verbosity(log_level)
#     transformers.utils.logging.enable_default_handler()
#     transformers.utils.logging.enable_explicit_format()

    # Setup logging [SOURCE: https://programtalk.com/python-more-examples/transformers.utils.logging.enable_default_handler/]
    logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        # level=logging.INFO if training_args.local_rank in [-1, 0] else logging.WARN
        level=logging.INFO,
    )
    transformers.utils.logging.enable_default_handler()
    transformers.utils.logging.enable_explicit_format()
    # Set the verbosity to info of the Transformers logger (on main process only):
    if is_main_process(training_args.local_rank):
        transformers.utils.logging.set_verbosity_info()
        
    # Log on each process the small summary:
    logger.info(
        f"Process rank: {training_args.local_rank}, device: {training_args.device}, n_gpu: {training_args.n_gpu}"
        + f"distributed training: {bool(training_args.local_rank != -1)}, 16-bits training: {training_args.fp16}"
    )
    logger.info(f"Training/evaluation parameters {training_args}")
    #print(f"\nTraining/evaluation parameters {training_args}\n")
    
    relation_representation = data_args.relation_representation
    
    model_list = model_args.model_list
    task_name = data_args.task_name
    dataset_name = data_args.dataset_name
    
    # Load relation and entity types.
    relation_type_file = os.path.join(data_args.dataset_dir, dataset_name)
    relation_type_file = os.path.join(relation_type_file, "relation_types.json")
    relation_types = json.load(open(relation_type_file))

    label_list = list(relation_types.keys())
        
    initial_output_dir = training_args.output_dir
    print(f"\n\n\n Initial Output Dir at first instance:{initial_output_dir}\n\n\n")
    
    if not os.path.exists(initial_output_dir):
        os.makedirs(initial_output_dir)
    
    # Here the author modifies the output_dir to be from being initial_output_dir to output_dir of model_name and relation_representation
    for model_name in model_list:
        model_args.model_name_or_path = model_name
        training_args.output_dir = initial_output_dir
        print(f"\n\n\n Initial Output Dir in for loop:{initial_output_dir}\n\n\n")
        training_args.output_dir = os.path.join(training_args.output_dir, dataset_name)
        training_args.output_dir = os.path.join(training_args.output_dir, model_name)
        training_args.output_dir = os.path.join(training_args.output_dir, relation_representation)
        
        if data_args.use_context != None:
            if data_args.use_context == "attn_based":
                training_args.output_dir += "_ac"

        # Set seed before initializing model.
        set_seed(training_args.seed)

        # Padding strategy
        if data_args.pad_to_max_length:
            padding = "max_length"
        else:
            # We will pad later in data collator, dynamically at batch creation, to the max sequence length in each batch
            padding = False
        
        def compute_metrics(p: EvalPrediction):
            print(f"\nRunning Training Predictions: {EvalPrediction}\n")
            pred, true = p.predictions, p.label_ids

            pred = np.argmax(pred, axis=1)
            true = true.flatten()

            pred = pred.tolist()
            true = true.tolist()
            
            # Remove ignored labels.
            # For ChemProt, ignore false labels. "CPR:false": "id": 0
            # For DDI, ignore false labels. "DDI-false": "id": 0
            # For TACRED, ignore no relation labels. "no_relation": "id": 0
            if any(x == dataset_name for x in ['ChemProt_BLURB', 'DDI_BLURB', 'TACRED']):
                cleaned_pred_true = [(p, t) for (p, t) in zip(pred, true) if t != 0]
                pred = [x[0] for x in cleaned_pred_true]
                true = [x[1] for x in cleaned_pred_true]
            
            # metrics ref: https://github.com/huggingface/datasets/tree/master/metrics
            a_m = load_metric("accuracy")
            p_m = load_metric("precision")
            r_m = load_metric("recall")
            f_m = load_metric("f1")
            
            if any(x == dataset_name for x in ["GAD_BLURB", "EU-ADR_BioBERT"]):
                average = "binary"
            else:
                average = "micro"
            
            a = a_m.compute(predictions=pred, references=true)
            p = p_m.compute(predictions=pred, references=true, average=average)
            r = r_m.compute(predictions=pred, references=true, average=average)
            f = f_m.compute(predictions=pred, references=true, average=average)
            
            return {"accuracy": a["accuracy"], "precision": p["precision"], "recall": r["recall"], "f1": f["f1"]}

        # Remove old output files except the cross-validation result file.
        if os.path.exists(training_args.output_dir):
            for f in os.listdir(training_args.output_dir):
                if os.path.isfile(os.path.join(training_args.output_dir, f)) and f != "predict_results_history.json":
                    os.remove(os.path.join(training_args.output_dir, f))
            
        # Get the number of datasets. If it's more than 1, it's a cross-validation dataset.
        num_of_datasets = 0
        for file in os.listdir(os.path.join(data_args.dataset_dir, data_args.dataset_name)):
            if file.startswith('train_') and file.endswith('.json'):
                num_of_datasets += 1
                print(f"*** No. of datasets: {num_of_datasets} ***")

        for dataset_num in range(num_of_datasets):
            logger.info("\n\n*** Dataset number: " + str(dataset_num) + " ***\n\n")

            # Load pretrained model and tokenizer
            #
            # In distributed training, the .from_pretrained methods guarantee that only one local process can concurrently
            # download model & vocab.
            config = AutoConfig.from_pretrained(
                model_args.config_name if model_args.config_name else model_args.model_name_or_path,
                num_labels=len(label_list),
                finetuning_task=task_name,
                cache_dir=model_args.cache_dir,
                revision=model_args.model_revision,
                use_auth_token=True if model_args.use_auth_token else None,
                
                # get attention outputs for attention based context.
                # ref: https://discuss.huggingface.co/t/output-attention-true-after-downloading-a-model/907
                output_attentions=True if data_args.use_context == "attn_based" else None,
            )
            
            # Explicitly set 'do_lower_case' since some models have a wrong case setting. (e.g., BioBERT, SciBERT)
            # HuggingFace ALBERT models and PubMedBERT are uncased.
            if config.model_type == "albert" or "PubMedBERT" in model_name:
                do_lower_case = True 
            else:
                do_lower_case = model_args.do_lower_case
            
            tokenizer_name_or_path = model_args.tokenizer_name if model_args.tokenizer_name else model_args.model_name_or_path
            # Use 'add_prefix_space' for GPT2, RoBERTa, DeBERTa
            if config.model_type in {"gpt2", "roberta", "deberta", "deberta-v2"}:
                tokenizer = AutoTokenizer.from_pretrained(
                    tokenizer_name_or_path,
                    cache_dir=model_args.cache_dir,
                    use_fast=True,
                    revision=model_args.model_revision,
                    use_auth_token=True if model_args.use_auth_token else None,
                    add_prefix_space=True,
                )
            else:
                # Set 'do_lower_case' when loading a tokenizer. It's not working to change the variable after it's loaded (i.e., tokenizer.do_lower_case = False).
                # GPT2, RoBERTa, DeBERTa don't have 'do_lower_case'. 
                tokenizer = AutoTokenizer.from_pretrained(
                    tokenizer_name_or_path,
                    cache_dir=model_args.cache_dir,
                    use_fast=True,
                    revision=model_args.model_revision,
                    use_auth_token=True if model_args.use_auth_token else None,
                    do_lower_case=do_lower_case,
                )
            
            max_seq_length = min(data_args.max_seq_length, tokenizer.model_max_length)
            
            ### TODO: delete this if not necessary (it doesn't affect the performance). 03-02-2022
            #
            # The default max length of BioBERT & DeBERTa is too large (1000000000000000019884624838656), so set it to 512.
            '''
            if isinstance(tokenizer, BertTokenizerFast) or isinstance(tokenizer, DebertaTokenizerFast) or isinstance(tokenizer, DebertaV2Tokenizer):
                max_seq_length = 512 
            '''
            
            # Remove any parent directories of the dataset if exist.
            dataset_name = dataset_name.rsplit('/', 1)[1] if len(dataset_name.rsplit('/', 1)) > 1 else dataset_name
            
            # Yashfinul Haque - 27/12/23
            logger.info(f"\n\nDataset being processed:{dataset_name}\n\n")
            if dataset_name in dataset_max_seq_length.keys():
                max_seq_length = dataset_max_seq_length[dataset_name]

            # Add special tokens for entity markers.
            #
            # Special tokens do not need to be lowercased for uncased models because they are basically all uppercased in the datasets. 03/30/2022
            # When special_tokens is set to False (default) in add_tokens(), it treats added tokens as normal tokens.
            # E.g., (added tokens: '@gene$', '@disease$') tokenizer.tokenize("@gene$, @disease$"), tokenizer.tokenize("@GENE$, @DISEASE$")
            #       -> the same output: ['@gene$', '@disease$']
            # When special_tokens is set to True in add_tokens(), it treats added tokens as special tokens.
            # E.g., (added tokens: '@gene$', '@disease$') tokenizer.tokenize("@gene$, @disease$"), tokenizer.tokenize("@GENE$, @DISEASE$")
            #       -> the different output: ['@gene$', '@disease$'], ['@', 'gene', '$', ',', '@', 'disease', '$']
            #
            # For some reason, add_tokens(special_tokens=True) doesn't update tokenizer.all_special_ids, tokenizer.all_special_tokens, ... 03/30/2022
            # Use add_special_tokens() instead.
            additional_special_tokens = []
            if relation_representation.startswith('EM'):
               additional_special_tokens.extend(entity_marker_special_tokens['EM'])
            
            # Add the special tokens that are used to replace entity names (entity anonymization or dummification). E.g,. ChemProt, DDI, GAD
            if dataset_name in dataset_special_tokens.keys():
                additional_special_tokens.extend(dataset_special_tokens[dataset_name])

            # Add additional special tokens at once. If they are added separately, then only tokens added later remain. 05/04/2022
            if len(additional_special_tokens) > 0:
                tokenizer.add_special_tokens({"additional_special_tokens": additional_special_tokens})
          
            model = BertForRelationClassification.from_pretrained(
                model_args.model_name_or_path,
                from_tf=bool(".ckpt" in model_args.model_name_or_path),
                config=config,
                cache_dir=model_args.cache_dir,
                revision=model_args.model_revision,
                use_auth_token=True if model_args.use_auth_token else None,
                
                # keyword parameters for RE
                relation_representation=relation_representation,
                use_context=data_args.use_context,
                tokenizer=tokenizer,
            )
                
            # Resize input token embeddings matrix of the model since new tokens have been added.
            # this funct is used if the number of tokens in tokenizer is different from config.vocab_size.
            model.resize_token_embeddings(len(tokenizer))

            # Loading a dataset from your local files.
            data_files = read_dataset(dataset_num, task_name, data_args)
            
            print(f"\n\n\nDataset Loading: {data_files}.\n\n\n")
            
            print(f"\nDataset Name: {dataset_name}.\n")
            # Yashfinul Haque - 27/12/23
            logger.info(f"\n\n\nProcessing {data_files}.\n\n\n")

            # This is a temporary code.
            # In BioInfer, some entities consist of separate tokens in a text, and the partial tokens are not properly tokenized by tokenizer.
            # To avoid the mismatch between entity index from data file and entity index from tokenized output, add the partial tokens to the tokenzier
            # so that the tokenizer properly catch partial tokens.
            # E.g., "GP IIIa" in "GPIIb-IIIa", "MEK 2" in "MEK1/2"
            if dataset_name == "BioInfer":
                partial_token_list = []
                for d in concatenate_datasets([data_files["train"], data_files["test"]]):
                    if len(d['relation'][0]['entity_1_idx']) > 1:
                        for (s, e) in d['relation'][0]['entity_1_idx']:
                            if d['text'][s-1] != ' ' or d['text'][e] != ' ':
                                partial_token = d['text'][s:e]
                                partial_token_list.extend(partial_token.split())

                partial_token_list = list(set(partial_token_list))
                tokenizer.add_tokens(partial_token_list)

                # Resize input token embeddings matrix of the model since new tokens have been added.
                # this funct is used if the number of tokens in tokenizer is different from config.vocab_size.
                model.resize_token_embeddings(len(tokenizer))

            dataset = featurize_data(data_files, 
                tokenizer, 
                padding, 
                max_seq_length, 
                relation_representation, 
                data_args.use_context,
            )

            train_dataset = dataset["train"]
            print(f"\nTraining Dataset: {train_dataset}\n")
            eval_dataset = dataset["validation"] if "validation" in dataset else None
            test_dataset = dataset["test"]
            
            # Data collator
            data_collator = DataCollatorForRelationClassification(tokenizer, pad_to_multiple_of=8 if training_args.fp16 else None)
            
            # Initialize our Trainer
            trainer = Trainer(
                model=model,
                #model_init=get_model,
                args=training_args,
                train_dataset=train_dataset if training_args.do_train else None,
                eval_dataset=eval_dataset if training_args.do_eval else None,
                tokenizer=tokenizer,
                data_collator=data_collator,
                compute_metrics=compute_metrics,
            )
                    
            # Detecting last checkpoint.
            last_checkpoint = None
            if os.path.isdir(training_args.output_dir and training_args.do_train and not training_args.overwrite_output_dir):
                last_checkpoint = get_last_checkpoint(training_args.output_dir)
                if last_checkpoint is None and len(os.listdir(training_args.output_dir)) > 0:
                    raise ValueError(
                        f"Output directory ({training_args.output_dir}) already exists and is not empty. "
                        "Use --overwrite_output_dir to overcome."
                    )
                elif last_checkpoint is not None:
                    logger.info(
                        f"Checkpoint detected, resuming training at {last_checkpoint}. To avoid this behavior, change "
                        "the `--output_dir` or add `--overwrite_output_dir` to train from scratch."
                    )
            
            # Training
            if training_args.do_train:
                checkpoint = None
                if training_args.resume_from_checkpoint is not None:
                    checkpoint = training_args.resume_from_checkpoint
                elif last_checkpoint is not None:
                    checkpoint = last_checkpoint
                train_result = trainer.train(resume_from_checkpoint=checkpoint)
                metrics = train_result.metrics
                max_train_samples = (
                    data_args.max_train_samples if data_args.max_train_samples is not None else len(train_dataset)
                )
                metrics["train_samples"] = min(max_train_samples, len(train_dataset))
                
                # Yashfinul Haque - 26/11/23
                trainer.save_model()  # Saves the tokenizer too for easy upload

                trainer.log_metrics("train", metrics)
                trainer.save_metrics("train", metrics)
                trainer.save_state()
                
                # After training completes, perform ONNX export
                torch.onnx.export(
                    model,
                    "To identify the regions of MCP-1 that contact its receptor, CCR2, we substituted all surface-exposed residues with alanine",
                    "bert_onnx_model.onnx",
                    opset_version=11,  # Adjust based on ONNX Runtime compatibility
                    do_constant_folding=True,  # Optional optimization
                    input_names=["input_ids", "attention_mask"],
                    output_names=["logits"],
                    dynamic_axes={
                        "input_ids": {0: "batch_size"},  # Handle variable batch sizes
                        "attention_mask": {0: "batch_size"}
                    }
                )
            
                eval_info = {"dataset_num": dataset_num, 
                             "seed": training_args.seed,
                             "epoch": training_args.num_train_epochs,
                             "per_device_train_batch_size": training_args.per_device_train_batch_size,
                             "n_gpu": torch.cuda.device_count() if torch.cuda.is_available() else 0,
                             "learning_rate": training_args.learning_rate,
                             "warmup_ratio": training_args.warmup_ratio,
                             "weight_decay": training_args.weight_decay}

            # Evaluation
            if training_args.do_eval:
                logger.info("*** Evaluate ***")
                
                if eval_dataset == None:
                    logger.info("***Evaluation Skipped***")
                    print("***Evaluation Skipped***")
                    break
                else:
                    metrics = trainer.evaluate(eval_dataset=eval_dataset)

                    max_eval_samples = (
                        data_args.max_eval_samples if data_args.max_eval_samples is not None else len(eval_dataset)
                    )
                    metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))

                    trainer.log_metrics("eval", metrics)

                    # Yashfinul Haque - 26/11/23
                    metrics["eval_info"] = eval_info
                    trainer.save_metrics("eval", metrics=metrics, combined=False)
            # Export to ONNX
            torch.onnx.export(
                model,  # Model to be exported
                train_dataset,  # Input tensors
                "PPI-RE-onnx.onnx",  # Output file path
                opset_version=11,  # Target ONNX version
                do_constant_folding=True,  # Optimize by folding constant values
                input_names=["input_ids", "attention_mask"],  # Specify input names
                output_names=["logits"],  # Specify output names
            )
        
            # Prediction
            if training_args.do_predict:
                logger.info("*** Predict ***")

                predictions, labels, metrics = trainer.predict(test_dataset, metric_key_prefix="predict")
                
                trainer.log_metrics("predict", metrics)
                trainer.save_metrics("predict", metrics, eval_info=eval_info)
                
                if data_args.save_predictions:
                    output_predict_file = os.path.join(training_args.output_dir, f"predict_outputs.txt")
                    if trainer.is_world_process_zero():
                        predictions = np.argmax(predictions, axis=1)
                        with open(output_predict_file, "a") as writer:
                            logger.info(f"***** Predict outputs *****")
                            if os.path.getsize(output_predict_file) == 0:
                                writer.write("index\tentity_1\tentity_2\ttext\tprediction\tlabel\n")
                            for index, (sample, item, label) in enumerate(zip(test_dataset, predictions, labels)):
                                item = label_list[item]
                                
                                # debug
                                if sample['labels'] != label:
                                    raise Exception("Label mismatch!!")
                                
                                ### TODO: for now, each sample has a single relation. Handle multiple relations in a sentence later.
                                label = label_list[label[0]]

                                input_ids = sample['input_ids']
                                sent = tokenizer.decode(input_ids)
                                sent = sent.replace('[CLS]', '').replace('[SEP]', '').strip()

                                def divide_chunks(l, n):
                                    for i in range(0, len(l), n): 
                                        yield l[i:i+n]
                                
                                ### TODO: for now, each sample has a single relation. Handle multiple relations in a sentence later.
                                e1_span_idx_list, e2_span_idx_list = list(divide_chunks(sample['relations'], 2))[0]

                                e1 = ' '.join([tokenizer.decode(input_ids[s:e]) for s, e in e1_span_idx_list])
                                e2 = ' '.join([tokenizer.decode(input_ids[s:e]) for s, e in e2_span_idx_list])

                                writer.write(f"{index}\t{e1}\t{e2}\t{sent}\t{item}\t{label}\n")
            

                                
        if training_args.do_predict:
            # Save the prediction results in a history file.
            result_file = os.path.join(training_args.output_dir, 'predict_results.json')
            f = open(result_file)
            
            txt = ''
            for line in f.readlines():
                line = line.strip()
                if line == '}{':
                    line = '}\n{'
                txt += line
            
            txt = txt.split('\n')
            
            entries = [json.loads(x) for x in txt]
            
            result_data = {}
            for entry in entries:
                for k, v in entry.items():
                    if k in result_data:
                        result_data[k].append(v)
                    else:
                        result_data[k] = [v]
            
            out_data = {}
            
            # Add a predix for average scores of CV datasets.
            key_prefix = "avg_" if num_of_datasets > 1 else ""
            
            out_data[key_prefix + "predict_f1"] = float(np.mean(result_data["predict_f1"]))
            out_data[key_prefix + "predict_precision"] = float(np.mean(result_data["predict_precision"]))
            out_data[key_prefix + "predict_recall"] = float(np.mean(result_data["predict_recall"]))
            out_data[key_prefix + "predict_accuracy"] = float(np.mean(result_data["predict_accuracy"]))
            out_data[key_prefix + "predict_loss"] = float(np.mean(result_data["predict_loss"]))
            out_data["epoch"] = result_data["epoch"][0]
            out_data["per_device_train_batch_size"] = result_data["per_device_train_batch_size"][0]
            out_data["learning_rate"] = str(result_data["learning_rate"][0])
            out_data["warmup_ratio"] = result_data["warmup_ratio"][0]
            out_data["weight_decay"] = result_data["weight_decay"][0]
            out_data["n_gpu"] = result_data["n_gpu"][0]
            out_data["num_of_datasets"] = num_of_datasets
            
            if num_of_datasets > 1:
                trainer.log_metrics(f"{num_of_datasets} folds cross-validation predict", out_data)
            
            out_file = os.path.join(training_args.output_dir, 'predict_results_history.json')
            with open(out_file, "a") as f:
                json.dump(out_data, f, indent=4, sort_keys=True)
            
        logger.info("*** Data iterations are done.  ***")
        

if __name__ == "__main__":
    main()
    
    

Overwriting run_re.py


# **Running Inference**

### Exporting the model to ONNX for Visualization